In [1]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, ConcatDataset, WeightedRandomSampler
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score
from tqdm.auto import tqdm
import random
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split 

# Đảm bảo tính lặp lại của kết quả
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
random.seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Kiểm tra thiết bị
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
NUM_FEATURE = 32
percent_data = 0.1
NUM_CLASSES = 3
PACKET_NUM = 20
client_lr = 1e-4
NUM_EPOCHS = 100
BATCH_SIZE = 16

In [3]:
domain2 = pd.read_feather(f'SOICT Data\\Data chia mới\\{NUM_FEATURE}\\Domain 2_{NUM_FEATURE}.feather')
domain1 = pd.read_feather(f'SOICT Data\\Data chia mới\\{NUM_FEATURE}\\Domain 1_{NUM_FEATURE}.feather')

In [4]:
def data_processing(df, NUM_FEATURES):
   y_train = df['Label']
   flow_id = df['flow_id']

   df = df/255

   X_train = df.drop(['Label', 'flow_id'], axis=1)
   X_train = X_train.to_numpy()

   X_train = X_train.reshape(-1,20, NUM_FEATURES)
   y_train = y_train.to_numpy()

   y_train = y_train.reshape(-1,20)[:,-1]
   return X_train, y_train

In [5]:
d2_x, d2_y = data_processing(domain2, NUM_FEATURE)
d1_x, d1_y = data_processing(domain1, NUM_FEATURE)

d1_train_x, d1_test_x, d1_train_y, d1_test_y = train_test_split(d1_x, d1_y, 
                                   random_state=42,  
                                   test_size=0.9,  
                                   shuffle=True) 

len(d1_train_y)

890

In [6]:
new_d1_train_x = []
new_d1_train_y = []

for label in np.unique(d1_train_y):
  indices = np.where(d1_train_y == label)[0]
  num_samples_to_select = int(len(indices) * percent_data)
  selected_indices = np.random.choice(indices, size=num_samples_to_select, replace=False)
  new_d1_train_x.extend(d1_train_x[selected_indices])
  new_d1_train_y.extend(d1_train_y[selected_indices])

d1_train_x = np.array(new_d1_train_x)
d1_train_y = np.array(new_d1_train_y)

np.unique(d1_train_y, return_counts=True)

(array([0, 1, 2], dtype=int64), array([16, 45, 27], dtype=int64))

In [7]:
def reshape_data(data):
  """Reshapes the input data to the desired shape [batch_size, 1, 20, 256]."""
  return data.reshape(1, PACKET_NUM, NUM_FEATURE)

class ETCDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return reshape_data(self.X[idx]), self.y[idx]

# Recreate the dataloaders with the updated ETCDataset
d1_train = ETCDataset(d1_train_x, d1_train_y)
d2_train = ETCDataset(d2_x, d2_y)
d1_dataloader = DataLoader(d1_train, batch_size=BATCH_SIZE, shuffle=True)
d2_dataloader = DataLoader(d2_train, batch_size=BATCH_SIZE, shuffle=True)

test_dataset = ETCDataset(d1_test_x, d1_test_y)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

for i, data in enumerate(d1_dataloader):
    inputs, labels = data
    print(inputs.shape)
    break

torch.Size([16, 1, 20, 32])


In [8]:
import torch.nn.functional as F

class CNNModel(nn.Module):
    def __init__(self, input_shape, num_classes=3):
        super(CNNModel, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=128, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(in_channels=128, out_channels=64, kernel_size=5, padding=2)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(in_channels=64, out_channels=32, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1)
        self.conv6 = nn.Conv2d(in_channels=32, out_channels=16, kernel_size=3, padding=1)

        self.flatten_dim = self._get_flatten_dim(input_shape)

        self.fc1 = nn.Linear(self.flatten_dim, 256)
        self.dropout = nn.Dropout(0.1)
        self.fc2 = nn.Linear(256, num_classes)

    def _get_flatten_dim(self, input_shape):
        with torch.no_grad():
            x = torch.zeros(1, 1, *input_shape)  # Tạo tensor dummy với kích thước input
            x = self.pool(F.relu(self.conv1(x)))
            x = self.pool(F.relu(self.conv2(x)))
            x = F.relu(self.conv3(x))
            x = self.pool(F.relu(self.conv4(x)))
            x = F.relu(self.conv5(x))
            x = self.pool(F.relu(self.conv6(x)))
            return x.numel()  # Flatten toàn bộ tensor còn lại

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = self.pool(F.relu(self.conv4(x)))
        x = F.relu(self.conv5(x))
        x = self.pool(F.relu(self.conv6(x)))

        x = torch.flatten(x, start_dim=1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [ ]:
class MultiClassTrAdaBoostCNN:
    def __init__(self, base_learner, n_estimators=10):
        """
        Khởi tạo lớp MultiClassTrAdaBoostCNN.
        
        Args:
            base_learner: Mô hình CNN cơ sở (base learner)
            n_estimators: Số lượng bộ học viên (mặc định là 10)
        """
        self.base_learner = base_learner
        self.n_estimators = n_estimators
        self.learners = []  # Danh sách các bộ học viên
        self.alphas = []    # Danh sách trọng số của các bộ học viên

    def fit(self, target_dataset, source_dataset):
        """
        Huấn luyện mô hình sử dụng dữ liệu mục tiêu và dữ liệu nguồn.
        
        Args:
            target_dataset: Tập dữ liệu mục tiêu (target domain)
            source_dataset: Tập dữ liệu nguồn (source domain)
        """
        n_target = len(target_dataset)
        n_source = len(source_dataset)
        total_samples = n_target + n_source
        K = NUM_CLASSES  # Số lớp (được định nghĩa trước)

        # Kết hợp tập dữ liệu
        combined_dataset = ConcatDataset([target_dataset, source_dataset])
        alpha_s = np.log(1 / (1 + np.sqrt(2 * np.log(n_target) / self.n_estimators)))
        # Khởi tạo trọng số mẫu
        beta = np.ones(total_samples) / total_samples
        
        for t in range(self.n_estimators):
            # Bước 1: Lấy mẫu có trọng số
            p_t = beta / beta.sum()
            sampler = WeightedRandomSampler(p_t, num_samples=total_samples, replacement=True)
            dataloader = DataLoader(combined_dataset, batch_size=BATCH_SIZE, sampler=sampler)

            # Bước 2: Huấn luyện bộ học viên cơ sở
            learner = self.base_learner(input_shape=(NUM_FEATURE, PACKET_NUM), num_classes=NUM_CLASSES).to(device)
            optimizer = optim.Adam(learner.parameters(), lr=1e-4)
            criterion = nn.CrossEntropyLoss()

            for epoch in tqdm(range(NUM_EPOCHS)):
                learner.train()
                for batch_idx, (data, target) in enumerate(dataloader):
                    data, target = data.float().to(device), target.long().to(device)
                    optimizer.zero_grad()
                    output = learner(data)
                    loss = criterion(output, target)
                    loss.backward()
                    optimizer.step()
                    
            learner.eval()
            predictions = []
            labels = []
            with torch.no_grad():
                eval_loader = DataLoader(combined_dataset, batch_size=BATCH_SIZE, shuffle=False)
                for data, target in eval_loader:
                    data, target = data.float().to(device), target.to(device)
                    output = learner(data)
                    loss = criterion(output, target)
                    val_loss += loss.item()
                    pred = torch.argmax(output, dim=1)
                    predictions.extend(pred.cpu().numpy())
                    labels.extend(target.cpu().numpy())

            # Tách dự đoán cho miền nguồn
            preds_source = predictions[n_target:]
            labs_source = labels[n_target:]

            # Tính lỗi trên miền nguồn
            beta_source = beta[n_target:]
            indicator_source = np.array([1 if p != l else 0 for p, l in zip(preds_source, labs_source)])
            eps_t = (beta_source * indicator_source).sum() / beta_source.sum()
            
            # Giới hạn lỗi để tránh bất ổn định số học
            eps_t = np.clip(eps_t, 1e-10, (K - 1) / K - 1e-10)

            # Bước 4: Tính trọng số của bộ học viên (alpha_t)
            alpha_t = np.log((1 - eps_t) / eps_t) + np.log(K - 1)
            
            # Bước 5: Tính yếu tố điều chỉnh
            C_t = K * (1 - eps_t)
            
            # Cập nhật trọng số cho miền mục tiêu và miền nguồn
            for i in range(total_samples):
                if i < n_target:
                    # Miền mục tiêu: cập nhật trọng số đơn giản
                    beta[i] = beta[i] * np.exp(alpha_t * (1 if predictions[i] != labels[i] else 0))
                else:
                    # Miền nguồn: áp dụng yếu tố điều chỉnh
                    beta[i] = C_t * beta[i] * np.exp(alpha_s * (1 if predictions[i] != labels[i] else 0))

            print(f"Iteration {t+1}/{self.n_estimators}, Source Error: {eps_t:.4f}")
            
            # Lưu trữ bộ học viên và trọng số của nó
            self.learners.append(learner)
            self.alphas.append(alpha_t)

    def predict(self, dataloader):
        """
        Dự đoán trên tập dữ liệu kiểm tra.
        
        Args:
            dataloader: DataLoader chứa dữ liệu kiểm tra
        
        Returns:
            Danh sách dự đoán cuối cùng
        """
        final_preds = []
        with torch.no_grad():
            for data, _ in dataloader:
                data = data.float().to(device)
                # Khởi tạo ma trận bỏ phiếu
                vote_matrix = torch.zeros(data.size(0), NUM_CLASSES, device=device)
                
                # Bỏ phiếu có trọng số từ tất cả các bộ học viên
                for alpha_t, learner in zip(self.alphas, self.learners):
                    learner.eval()
                    outputs = learner(data)
                    preds = torch.argmax(outputs, dim=1)
                    
                    # Cộng dồn phiếu có trọng số
                    for i in range(data.size(0)):
                        vote_matrix[i, preds[i]] += alpha_t
                
                # Dự đoán cuối cùng dựa trên phiếu tối đa
                final_preds_batch = torch.argmax(vote_matrix, dim=1)
                final_preds.extend(final_preds_batch.cpu().numpy())
                
        return final_preds

# Ví dụ sử dụng
model = MultiClassTrAdaBoostCNN(CNNModel, n_estimators=10)
model.fit(d1_train, d2_train)

  0%|          | 0/100 [00:00<?, ?it/s]

Iteration 1/10, Source Error: 0.0319


  0%|          | 0/100 [00:00<?, ?it/s]

Iteration 2/10, Source Error: 0.0163


  0%|          | 0/100 [00:00<?, ?it/s]

...

Iteration 8/10, Source Error: 0.0010


  0%|          | 0/100 [00:00<?, ?it/s]

Iteration 9/10, Source Error: 0.0044


  0%|          | 0/100 [00:00<?, ?it/s]

Iteration 10/10, Source Error: 0.0002


In [12]:
predictions = model.predict(test_dataloader)
y_true = d1_test_y
y_pred = predictions
report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1538
           1       0.83      0.76      0.79      4190
           2       0.62      0.71      0.66      2282

    accuracy                           0.79      8010
   macro avg       0.81      0.82      0.82      8010
weighted avg       0.80      0.79      0.79      8010



In [11]:
with open(f'Results/{NUM_FEATURE}/classification_report_TrAdaBoost_{NUM_FEATURE}_{percent_data}.txt', 'w') as f:
    f.write(report)
